In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# data source: https://surfdrive.surf.nl/files/index.php/s/CZWfWQp3VKGKa8m
df = pd.read_csv('data_with_ports.csv.gz', sep='|', compression='gzip')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (15,17,19,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# parse string to datetime format
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['start_time'] = pd.to_datetime(df['start_time'], format='%Y-%m-%dT%H:%M:%S+09:00')
df['stop_time'] = pd.to_datetime(df['stop_time'], format='%Y-%m-%dT%H:%M:%S+09:00')

In [4]:
# remove reserved ips

reserved_ips = [
    "0.",       # software
    "10.",      # private network
    "100.",     # private network
    "169.254.", # subnet
    "172.",     # private network
    "192."      # private network
]

for ips in reserved_ips:
    df = df[~(df.org.isna() & df.target_ip.str.startswith(ips))]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5719250 entries, 0 to 5721431
Data columns (total 35 columns):
target_ip                 object
date                      datetime64[ns]
sensor_id                 object
service                   object
start_time                datetime64[ns]
stop_time                 datetime64[ns]
duration                  float64
packets                   int64
raw_country               object
raw_as                    object
raw_hostname              object
udp_port_list             object
pyasn_as                  float64
pyasn_as_bgp_size         float64
cc                        object
region                    object
is_oecd                   float64
as_type                   object
as_type_confidence        float64
tg_op                     object
org                       object
org_range                 object
org_rangesize             float64
org_ipsize_seen           float64
org_domainsize_seen       float64
org_ipsize_seen_shared    float

In [6]:
df.as_type.unique()

array(['isp-broadband', nan, 'isp-other', 'isp-mobile',
       'other-intermediary', 'hosting', 'non-intermediary', 'gov', 'edu'],
      dtype=object)

In [7]:
df.org_tag.unique()

array([nan, 'other', 'isp-mobile', 'isp-broadband', 'isp-other',
       'hosting', 'cdn', 'edu', 'gov'], dtype=object)

### Major hosting providers

In [8]:
# a list of major hosting providers
hosts = ["godaddy", "amazon", "google", "1&1", "squarespace", "hostgator", "ovh", "hetzner", "softlayer", "liquid web"]

In [9]:
majorhps = (df[df.org.str.contains("|".join(hosts), case=False, na=False)]
              [["org", "org_tag", "as_type", "as_ipsize_seen"]]
              .drop_duplicates()
              .sort_values(by=["org"]))

In [10]:
majorhps.to_csv("major-hosting-providers.csv")